## Cards

In [2]:
import os
import json
import uuid
import pandas as pd
import datetime
import shutil
import csv

lookup_path = './lookup/'
source_path = './out/'
mappings_path = './mappings/'
#save_path = '../../../../digipolis-arches-shoku-pkg/source/'
source_file_name = './out/monuments.json'

## Functions

In [3]:
def resource_identifier(source_id):
    return str(uuid.uuid5(uuid.NAMESPACE_DNS, source_id))


def generate_uri(source_id):
    return str(f'https://www.geonames.org/{source_id}')


def coordinates_geo_json(source_value): 
    
    return f'POINT {source_value}'.replace(",", "")


def coordinates_wkt(source_value):    
    return f'{source_value}'.replace(",", "")


def get_constants(node_mapping):
    constant_dict = {}
    constants = node_mapping['constants']
    for constant in constants:
        for constant_label, value in constant.items():
            constant_dict[constant_label] = value
    return constant_dict


def lookup(code, lookup_file):
    lookup_df = pd.read_csv(lookup_path + lookup_file)
    try:
        concept_name = lookup_df[lookup_df['code'] == code]['concept'].iloc[0]
        return concept_name
    except:
        print(f'{code} NOT FOUND IN {lookup_file}')
        return f'{code} NOT FOUND IN {lookup_file}'


## Save csv

In [4]:
def save_mapped_set(source_df, _package, _resource_model, _card):
    
    package = _package.split('_')[1]     
    
    file_name = '%s/%s/%s/%s.csv' % (save_path, package, _resource_model, _card)
    source_df.to_csv(file_name, index=False)    
    del source_df
    return 'Saved OK'

In [5]:
def related_resource(source_value):
    resourceid = str(uuid.uuid5(uuid.NAMESPACE_DNS, source_value))
    resourceXresourceId = str(uuid.uuid5(uuid.NAMESPACE_DNS, source_value + 'relation'))
    relation = '[{"resourceId": "%s", "ontologyProperty": "", "resourceXresourceId": "%s", "inverseOntologyProperty": ""}]' % (resourceid, resourceXresourceId)
    return relation

## Places

In [6]:
place_path = '../geonames/out/single_mapped.csv'
#places = csv.DictReader(open(f'{place_path}', encoding='utf-8-sig'), delimiter=',')
placed_df = pd.read_csv(place_path)

def match_geonames(obj_city):

    if obj_city == "'s-Hertogenbosch":
        geonames_place = 'c505eba4-4331-5770-b84d-0f424b92a494'
        return geonames_place
    elif obj_city == "Heeze":
        geonames_place = '9bed8115-dba2-56b4-9f65-3ad03690d46a'
        return geonames_place
    elif obj_city == "Bavel":
        geonames_place = '933453cb-1ee2-52f8-bf28-683a2704e1c0'
        return geonames_place
    elif obj_city == "Baarle-Hertog":
        geonames_place = '8275b9e6-029d-44cd-b4c0-3807682ec2a2'
        return geonames_place
    
    
    else:
        geonames_place = placed_df[(placed_df['type'] == 'populated place') & (placed_df['name_content'] == obj_city)]
        try:
            row = geonames_place.iloc[0]
            #print(row['ResourceID'])
            return row['ResourceID'] 
        except:   
            print(obj_city)
            return None
        
    
     

In [7]:
def make_places(source_list, node_mappings, graph, dataset, card_name, card_order):
    
    obj_dict = {}
    obj_list = []
    
    for obj in source_list:
        if 'placeURI' in obj.keys():
        
            obj_dict['ResourceID'] = obj['ResourceID']
            obj_dict['card'] = 'place'
            obj_dict['card_order'] = '5'
            
            try:
                obj_dict['location_address_street'] = obj['placeURI']['PostalAddress']['streetAddress']
            except:
                pass
            obj_dict['location_address_city'] = obj['placeURI']['PostalAddress']['addressLocality']
            try:
                obj_dict['location_coordinates_geo_json'] = obj['latLong']
            except:
                pass
            
            geonames_resource = match_geonames(obj['placeURI']['PostalAddress']['addressLocality'])
            
            resourceXresourceId = str(uuid.uuid5(uuid.NAMESPACE_DNS, obj['placeURI']['PostalAddress']['addressLocality'] + 'relation'))
            obj_dict['location_place'] = '[{"resourceId": "%s", "ontologyProperty": "", "resourceXresourceId": "%s", "inverseOntologyProperty": ""}]' % (geonames_resource, resourceXresourceId)
            if geonames_resource:
               
                
                obj_list.append(obj_dict)
                obj_dict = {}
        
    source_df = pd.DataFrame(obj_list)
    number_of_records = len(obj_list)
        
    if number_of_records > 0:                       
        #save_it = save_mapped_set(source_df, package, resource_model, card)
        source_df.to_csv(f'out/buildings/places.csv', index =False)
        #return '%s saved: %s' % (card_name, number_of_records)
        return source_df


#relation = '[{"resourceId": "%s", "ontologyProperty": "", "resourceXresourceId": "%s", "inverseOntologyProperty": ""}]' % (usr_dict['ResourceID_mat'] , uuid.uuid4())


## Generate dataset

In [9]:
%%time

with open(mappings_path + 'mappings_places.json') as f:
    mappings_dict = json.load(f)  
    
    record_dict = {}
    record_list = []

    places_list = []

    name = mappings_dict['name']
    graph = mappings_dict['graph']
    print('-', name)
    print('  -', graph)
    
    for cards, cards_values in mappings_dict['mappings'].items():  
        card_name = cards
        for item,item_value in cards_values.items():
            if item == 'dataset': 
                dataset = item_value
            if item == 'card_order':
                card_order = item_value
            if item == 'filter':
                card_filter = item_value    
            if item == 'node_mappings':
                node_mappings = item_value    
           
        node_mappings_keys = [item['from'] for item in node_mappings]
        #records = csv.DictReader(open(f'{source_path}{dataset}', encoding='utf-8-sig'), delimiter=',')
        with open(source_file_name) as f:
            records = json.load(f)

            _landmarks = records['records']['record']

            for _record in _landmarks:                 ## add filter from mapping!  
                record = _record['LandmarksOrHistoricalBuildings']
                record_dict['ResourceID'] = str(uuid.uuid5(uuid.NAMESPACE_DNS, record['id']))
                record_dict['card'] = card_name
                record_dict['card_order'] = card_order

                for mapping_key in node_mappings_keys:
                    if mapping_key in record.keys():
                        if len(record[mapping_key]) > 0:                                
                            record_dict[mapping_key] = record[mapping_key]
                    
                record_list.append(record_dict)                            
                record_dict = {}
            
            if card_name == 'places':                
                places_list.extend(record_list)                    
            
            record_list = []
            
            
    if len(places_list) > 0:
        Places = make_places(places_list, node_mappings, graph, dataset, card_name, card_order)
        #print('    -', "Places", len(Places))
        
#    if len(alternate_name_list) > 0:
#        AlternateName = make_mapping(alternate_name_list, mappings, graph, dataset, card_name, card_order)
#        print('    -', "AlternateName", len(AlternateName))

#        #if len(digital_reference_list) > 0:
#        #    DigitalReference = make_mapping(digital_reference_list, mappings, graph, dataset, card_name, card_order)
#        #    print('    -', "DigitalReference", len(DigitalReference))


print('---------') 
print(datetime.datetime.now())
#print(json.dumps(Places, indent=2))
#%reset -f 
Places

- brabantse_gebouwen
  - Building
Aalst
Heusden (Gemeente Asten)
Heusden (Gemeente Heusden)
Zevenbergen (Gemeente Moerdijk)
St. Willebrord
Meerveldhoven
St. Willebrord
Hoeven (Gemeente Halderberge)
Hoeven (Gemeente Halderberge)
Stiphout
Zevenbergen (Gemeente Moerdijk)
Zevenbergen (Gemeente Moerdijk)
Oosteind (Gemeente Oosterhout)
St. Willebrord
Meerveldhoven
Wernhoutsburg
Heusden (Gemeente Asten)
Velp (Gemeente Land van Cuijk)
Velp (Gemeente Land van Cuijk)
Velp (Gemeente Land van Cuijk)
Waalre-dorp
Achel
Heusden (Gemeente Heusden)
Velp (Gemeente Land van Cuijk)
Wernhoutsburg
St. Willebrord
St. Willebrord
Zevenbergen (Gemeente Moerdijk)
Hamont
Hoogstraten
Hoogstraten
Waalre-dorp
Aalst
Waalre-dorp
Waalre-dorp
Aalst
Aalst
---------
2025-01-14 15:07:02.316778
CPU times: user 290 ms, sys: 14.4 ms, total: 304 ms
Wall time: 308 ms


,ResourceID,card,card_order,location_address_street,location_address_city,location_coordinates_geo_json,location_place
0,b746c24c-7a94-527b-8428-3b4bf01e5cef,place,5,Oude Peelstraat 1,Helenaveen,Point(5.9188 51.3885),"[{""resourceId"": ""547d5215-1415-55cc-b650-9bff1..."
1,4848283e-669b-5d45-82c3-fc731b5603ed,place,5,Strijpsestraat 144,Eindhoven,Point(5.4575 51.4409),"[{""resourceId"": ""e39e761f-1486-5098-a3c4-3f164..."
2,1dcddae3-52f4-5595-81cb-a3c6934fb032,place,5,Kerkstraat 20,Overlangel,Point(5.6731 51.7755),"[{""resourceId"": ""9959baf7-fb1a-5293-91e5-6ceec..."
3,4615c838-ac97-55b8-b8c0-0a23ada086e1,place,5,Hamstraat 1,Huisseling,Point(5.641 51.7878),"[{""resourceId"": ""6d49103a-1df1-5f30-a579-f3f88..."
4,85fd98ea-be01-5187-9254-6ecf4b9d4385,place,5,Antoon Coolenplein 3,Lith,Point(5.4389 51.8067),"[{""resourceId"": ""0ca3cb5a-81dc-5f29-9ce7-4156c..."
...,...,...,...,...,...,...,...
923,77eff9d0-0777-5852-a4fa-8a89b10654a2,place,5,Rijksweg 121,Rijen,Point(4.9154 51.5764),"[{""resourceId"": ""6b7e5b08-6bc9-52d3-ae22-579c0..."
924,ca6219e8-e960-5554-9459-7476b8e81bf0,place,5,Molenstraat 32a,Zeeland,Point(5.672 51.6977),"[{""resourceId"": ""91307e12-2a7d-5e76-a6e5-ad066..."
925,c9ae6740-f457-59a8-b253-6a3663dd12ee,place,5,Kerkstraat,Zeeland,Point(5.6739 51.6974),"[{""resourceId"": ""91307e12-2a7d-5e76-a6e5-ad066..."
926,64652f85-55c5-549d-a739-f52aaf3a506c,place,5,Voederheil 3,Zeeland,Point(5.6788 51.7062),"[{""resourceId"": ""91307e12-2a7d-5e76-a6e5-ad066..."
